# 사용자 정의 모델링
* `tf.keras.Model` 클래스를 상속 받아서 모델을 구현할 수 있다.
  * 생성자 `__init__` 함수에서는 사용할 레이어를 정의합니다.
  * `call()` 함수에서는 `__init__`에서 정의한 레이어를 엮어줍니다.

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):
  ## __init__ 에서는 사용할 레이어들을 "정의"만 한다.
  def __init__(self):
    super(MyModel, self).__init__() # 강제로 부모 클래스의 생성자를 호출해서 부모클래스의 모든 내용을 사용할 수 있도록 함.
    # 아주 간단한 FCN 구현하기
    self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  # call 에서는 __init__에서 정의한 레이어를 엮어준다. 결과를 "반드시" 리턴해야 한다.
  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    out = self.d2(x)

    return out

# 학습 루프 정의하기

In [ ]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
  """
    model : 학습대상 모델
    images : 학습에 사용할 이미지 텐서
    labels : 이미지에 맞는 레이블 텐서
    loss_object : 손실함수
    optimizer : 최적화 함수
    train_loss : 훈련 과정 Loss 평가 지표 설정
    train_accuracy : 훈련 과정 accuracy 평가 지표 설정
  """

  with tf.GradientTape() as tape:
    prediction = model(images)
    loss = loss_object(labels, prediction)

  grads = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, prediction)

# 테스트 루프 정의하기

In [ ]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuarcy):
  prediction = model(images)

  t_loss = loss_object(labels, prediction)

  test_loss(t_loss)
  test_accuracy(labels, prediction)

# 데이터 세트 로딩 및 전처리

In [ ]:
mnist = tf.keras.datasets.mnist

# MNIST 데이터세트 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 픽셀 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0

# 이미지 채널 차원 추가
X_train = X_train[..., tf.newaxis]
X_test  = X_test[..., tf.newaxis]

# Tensorflow Dataset 만들기
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

11493376/11490434 [==============================] - 0s 0us/step


# 모델 생성 (네트워크 생성)

In [ ]:
model = MyModel()

# 손실함수, 최적화 선택하기

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer   = tf.keras.optimizers.Adam()

# 알고리즘 평가지표 설정 ( performance metrics )

In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuarcy")

# 학습 진행하기

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):

  # 훈련
  for images, labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
  
  # 테스트
  for test_images, test_labels in test_ds:
    test_step(model, images, labels, loss_object, test_loss, test_accuracy)

  template = "Epoch {}, Train Loss : {:.3f}, Train Accuracy : {:.3f} 😀 Test Loss : {:.3f}, Test Accuracy : {:.3f}"
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,

                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Train Loss : 0.254, Train Accuracy : 92.870 😀 Test Loss : 0.242, Test Accuracy : 93.750
Epoch 2, Train Loss : 0.183, Train Accuracy : 94.773 😀 Test Loss : 0.144, Test Accuracy : 96.875
Epoch 3, Train Loss : 0.148, Train Accuracy : 95.748 😀 Test Loss : 0.097, Test Accuracy : 97.917
Epoch 4, Train Loss : 0.125, Train Accuracy : 96.360 😀 Test Loss : 0.077, Test Accuracy : 98.438
Epoch 5, Train Loss : 0.109, Train Accuracy : 96.828 😀 Test Loss : 0.065, Test Accuracy : 98.750
Epoch 6, Train Loss : 0.096, Train Accuracy : 97.185 😀 Test Loss : 0.055, Test Accuracy : 98.958
Epoch 7, Train Loss : 0.086, Train Accuracy : 97.472 😀 Test Loss : 0.049, Test Accuracy : 99.107
Epoch 8, Train Loss : 0.078, Train Accuracy : 97.708 😀 Test Loss : 0.044, Test Accuracy : 99.219
Epoch 9, Train Loss : 0.071, Train Accuracy : 97.912 😀 Test Loss : 0.041, Test Accuracy : 99.306
Epoch 10, Train Loss : 0.065, Train Accuracy : 98.074 😀 Test Loss : 0.037, Test Accuracy : 99.375


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
